These are the packages we need:

In [1]:


import sympy as sp
import numpy as np
from itertools import combinations_with_replacement as itTuples
import os.path

Generates al possible combination --of length k-- from a List of elements

In [2]:
def Tuples(List,k):
    #Generates al possible combination --of length k-- from a List of elements
    return list(itTuples(List,k))

MatrixProd([list1,list2,list3,...]) gives np.dot(list1,np.dot(list2,np.dot(list3,...))
 

In [3]:
def MatrixProd(a):
    n=len(a)-1
    if n!=0:
        return np.dot(MatrixProd(a[:n]),a[n])
    else:
        return a[0]
    

Calculates the derivative of L with respect to a: $\frac{dL}{da}$.

If a is a list, ti calculates the derivative $\frac{d^{n}L}{da_{1}da_{2}...da_{n}} $

In [4]:
def Deriv(L,a):
 
    try:  
        n=len(a)-1
        if n>=0:
            return sp.diff(Deriv(L,a[:n]),a[n])
        else:
            return L
    except:
        return sp.diff(L,a)

Define the paricles, parameters of the model for SU(DimN).

If Gauge='un', the G^{0}, G^{+} and G^{-} are not defined.

Also, this function defines the various usefun rules for substitution needed, such as subs0, which 
sets all fields to 0 (needed for idendifying the vertices and minimizing the potential).

In [5]:
def Definitions(DimN, Gauge):

    
    global gauge, dimN
    global dimRange, indexRange, mPhi, mPhip, v, vPhi, muH, lamH, lamHPhi, lamPhi
    global Gp, H0, Gm, H0t, h, G0, H, Ht, Phi, Phit, chi, rho, phi, s
    global Fields, sqrt2, subs0, subsvev, subsexpand
    
    dimN=DimN
    gauge=Gauge
    
    dimRange=range(1,dimN+1);
    dimRangeM=range(dimN-1)
    indexRange=range(0,dimN);

    sqrt2=sp.sqrt(2);

    
    mPhi=np.array( sp.symbols('mPhi(1:{})(1:{})'.format(str(dimN+1),str(dimN+1))   )            ).reshape(dimN,dimN)
    mPhip=np.array( sp.symbols('mPhip(1:{})(1:{})'.format(str(dimN+1),str(dimN+1))   )          ).reshape(dimN,dimN)
    
    #make mPhi symmetric (faster than np.triu(mPhi,+1).T+np.triu(mPhi))
    for i in range(dimN):
        for j in range(i+1,dimN):
            mPhi[j][i]=mPhi[i][j]
    
    #make mPhip hermitian (faster than np.conjugate(np.triu(mPhi,+1).T)+np.triu(mPhi))
    for i in range(dimN):
        for j in range(i+1,dimN):
            mPhip[j][i]=sp.conjugate(mPhip[i][j])
    #make the diagonal  real         
    for i in range(dimN):
        exec( 'mPhip[{}][{}]=sp.Symbol(  \'mPhip{}{}\'  ,real=True)'.format(str(i),str(i),str(i+1),str(i+1)) )

        

    
    Phi = sp.symbols('Phi1:{}'.format(str(dimN+1)))
    Phit = sp.symbols('Phi1:{}t'.format(str(dimN+1)))

    if gauge=='un':
        H0, H0t=sp.symbols('H0, H0t')
        H = [0,H0];
        Ht = [0, H0t];
    else:    
        H0,H0t,Gp,Gm=sp.symbols('H0,H0t,Gp,Gm')
        H = [Gp,H0];
        Ht = [Gm, H0t];

    
    
    ##################--Declare symbols for expaned scalars 
    phi = sp.symbols('phi1:{}'.format(str(dimN)))
    s = sp.symbols('s1:{}'.format(str(dimN)))
    h, G0 , chi, rho=sp.symbols('h G0 chi rho')

    
    
    v=sp.Symbol('v',positive=True);
    vPhi=sp.Symbol('vPhi',positive=True);
    muH=sp.Symbol('muH');
    lamH=sp.Symbol('lamH',real=True);
    lamHPhi=sp.Symbol('lamHPhi',real=True);
    lamPhi=sp.Symbol('lamPhi',real=True);



   

    
    #Expand the fields at their vevs
    if gauge=='un':
        subsexpand =np.array(\
                     [(H0,(h+v)/sqrt2 ),(H0t,(h+v)/sqrt2 ),\
                      (Phi[dimN-1],(rho+ sp.I*chi+vPhi)/sqrt2 ),\
                      (Phit[dimN-1],(rho-sp.I*chi+vPhi)/sqrt2 )]+ \
                     [(Phi[i], (phi[i]+sp.I*s[i])/sqrt2   ) for i in dimRangeM]+\
                     [(Phit[i],(phi[i]-sp.I*s[i])/sqrt2)  for i in dimRangeM])
                
        Fields=np.array(sp.flatten([h,rho,s,chi,phi]))        
    else:
        subsexpand = np.array(\
                      [(H0,(h+sp.I*G0+v)/sqrt2 ),(H0t,(h-sp.I*G0+v)/sqrt2 ),\
                      (Phi[dimN-1], (rho+sp.I*chi+vPhi)/sqrt2 ),\
                      (Phit[dimN-1],(rho-sp.I*chi+vPhi)/sqrt2 )]+ \
                     [(Phi[i], (phi[i]+sp.I*s[i])/sqrt2) for i in dimRangeM]+\
                     [(Phit[i],(phi[i]-sp.I*s[i])/sqrt2) for i in dimRangeM])
        
        Fields=np.array(sp.flatten([h,rho,s,chi,phi,G0,Gp,Gm]))
        
        
    subsvev = np.array(\
                      [(H0,v/sqrt2 ),(H0t,v/sqrt2 ),\
                       (G0,0),(Gm,0),(Gp,0),\
                      (Phi[dimN-1], vPhi/sqrt2 ),\
                      (Phit[dimN-1],vPhi/sqrt2 )]+ \
                     [(Phi[i], 0) for i in dimRangeM]+\
                     [(Phit[i],0) for i in dimRangeM])
    subs0=[ (i, 0) for i in Fields]


Should run after Definitions(DimN,Gauge)!
Since all parameters, fields and rules are global parameters in Definitions,
GetLagrangian() takes them and calculates the Potential and Lagrangian.
Here we define the substitution rules for the minimization of the potential.
The variable constV is the constant value of the potential, which helps us test the minimization conditions.

In [6]:
def GetLagrangian():

    
    global V, LMassInt, constV, subsmin
    
    mPhi2=[[i**2for i in x] for x in mPhi]
    mPhi2C=[[sp.conjugate(i)**2for i in x] for x in mPhi]
    mPhip2=[[i**2for i in x] for x in mPhip]

    V0=-muH**2/2*MatrixProd([H,Ht])+lamH/2*MatrixProd([H,Ht])**2+lamPhi/2*MatrixProd([Phi,Phit])**2\
    +lamHPhi*MatrixProd([H,Ht])*MatrixProd([Phi,Phit] );

    Vsoft=MatrixProd([Phi,mPhi2,Phi])+MatrixProd([Phit,mPhi2C,Phit])+MatrixProd([Phit,mPhip2,Phi])


    V=(V0+Vsoft)#.subs(subsexpand)
    
    subsmin= [ (mPhi[i][dimN-1], sp.I/sqrt2*mPhip[dimN-1][i] ) for i in range(0,dimN-1)]+ \
    [(muH, sp.sqrt(v**2*lamH + vPhi**2*lamHPhi)),(mPhip[dimN-1][dimN-1],\
    -sp.sqrt(-2*lamHPhi*v**2 - 2*lamPhi*vPhi**2 - 8*mPhi[dimN-1][dimN-1]**2)/2),
    (sp.conjugate(mPhi[dimN-1][dimN-1])**2,mPhi[dimN-1][dimN-1]**2 )]

    
    constV=sp.simplify((V.subs(subsmin).subs(subsvev)) )
    
    
    LMassInt = -((V.subs(subsmin)).subs(subsexpand));

# For subsmin:

there are four --for the SU(2)-- independent equations for the minimization of the potential.

In [7]:
Definitions(2,'feyn')
GetLagrangian()

for i in Fields:
    print sp.simplify(sp.expand_complex(Deriv(V.subs(subsexpand),i ).subs(subs0)) )

v*(lamH*v**2 + lamHPhi*vPhi**2 - re(muH)**2 - 2*I*re(muH)*im(muH) + im(muH)**2)/2
vPhi*(lamHPhi*v**2 + lamPhi*vPhi**2 + 2*mPhip22**2 + 4*re(mPhi22)**2 - 4*im(mPhi22)**2)/2
2*vPhi*(-2*re(mPhi12)*im(mPhi12) + re(mPhip12)*im(mPhip12))
-4*vPhi*re(mPhi22)*im(mPhi22)
vPhi*(2*re(mPhi12)**2 + re(mPhip12)**2 - 2*im(mPhi12)**2 - im(mPhip12)**2)
0
0
0


# For subsmin:

there are five --for the SU(3)-- independent equations for the minimization of the potential.

In [8]:
Definitions(3,'feyn')
GetLagrangian()

for i in Fields:
    print sp.simplify(sp.expand_complex(Deriv(V.subs(subsexpand),i ).subs(subs0)) )

v*(lamH*v**2 + lamHPhi*vPhi**2 - re(muH)**2 - 2*I*re(muH)*im(muH) + im(muH)**2)/2
vPhi*(lamHPhi*v**2 + lamPhi*vPhi**2 + 2*mPhip33**2 + 4*re(mPhi33)**2 - 4*im(mPhi33)**2)/2
2*vPhi*(-2*re(mPhi13)*im(mPhi13) + re(mPhip13)*im(mPhip13))
2*vPhi*(-2*re(mPhi23)*im(mPhi23) + re(mPhip23)*im(mPhip23))
-4*vPhi*re(mPhi33)*im(mPhi33)
vPhi*(2*re(mPhi13)**2 + re(mPhip13)**2 - 2*im(mPhi13)**2 - im(mPhip13)**2)
vPhi*(2*re(mPhi23)**2 + re(mPhip23)**2 - 2*im(mPhi23)**2 - im(mPhip23)**2)
0
0
0


# For subsmin:

there are eight --for the SU(6)-- independent equations for the minimization of the potential.

In [9]:
Definitions(6,'feyn')
GetLagrangian()

for i in Fields:
    print sp.simplify(sp.expand_complex(Deriv(V.subs(subsexpand),i ).subs(subs0)) )

v*(lamH*v**2 + lamHPhi*vPhi**2 - re(muH)**2 - 2*I*re(muH)*im(muH) + im(muH)**2)/2
vPhi*(lamHPhi*v**2 + lamPhi*vPhi**2 + 2*mPhip66**2 + 4*re(mPhi66)**2 - 4*im(mPhi66)**2)/2
2*vPhi*(-2*re(mPhi16)*im(mPhi16) + re(mPhip16)*im(mPhip16))
2*vPhi*(-2*re(mPhi26)*im(mPhi26) + re(mPhip26)*im(mPhip26))
2*vPhi*(-2*re(mPhi36)*im(mPhi36) + re(mPhip36)*im(mPhip36))
2*vPhi*(-2*re(mPhi46)*im(mPhi46) + re(mPhip46)*im(mPhip46))
2*vPhi*(-2*re(mPhi56)*im(mPhi56) + re(mPhip56)*im(mPhip56))
-4*vPhi*re(mPhi66)*im(mPhi66)
vPhi*(2*re(mPhi16)**2 + re(mPhip16)**2 - 2*im(mPhi16)**2 - im(mPhip16)**2)
vPhi*(2*re(mPhi26)**2 + re(mPhip26)**2 - 2*im(mPhi26)**2 - im(mPhip26)**2)
vPhi*(2*re(mPhi36)**2 + re(mPhip36)**2 - 2*im(mPhi36)**2 - im(mPhip36)**2)
vPhi*(2*re(mPhi46)**2 + re(mPhip46)**2 - 2*im(mPhi46)**2 - im(mPhip46)**2)
vPhi*(2*re(mPhi56)**2 + re(mPhip56)**2 - 2*im(mPhi56)**2 - im(mPhip56)**2)
0
0
0


# So, There  are N+2 conditions for the minimization of the potential (given by subsmin)!

The function IdentifyInteractions() finds all two-, three, and four-point interaction terms of the Largrangian

In [10]:
def IdentifyInteractions():
    global TwoPoint, ThreePoint, FourPoint,TypeOfVert
    
    TwoPoints = Tuples(Fields, 2)
    ThreePoints = Tuples(Fields, 3)
    FourPoints = Tuples(Fields, 4)
    
    TypeOfVert=['TwoPoint','ThreePoint','FourPoint']

    LMassIntfinal=0
    for TypeV in TypeOfVert:
        print 'Calculating '+TypeV+' interactions...'
        exec(TypeV+'=[]')
        
        for i in eval(TypeV+'s'):
            SymF=np.product([ sp.factorial(i.count(j)) for j in set(i)])
            tmpval=sp.simplify(1/SymF*Deriv(LMassInt,i).subs(subs0))
            if tmpval!=0:
                eval(TypeV+'.append([i, tmpval, SymF])')
            
        LMassIntfinal+=np.sum([ np.product(tmpi[0])*tmpi[1] for tmpi in eval(TypeV)])
    
    TwoPoint=eval('TwoPoint')
    ThreePoint=eval('ThreePoint')
    FourPoint=eval('FourPoint')

The function prtcls, gets a list of particles (prts) and returns a sorted list of them (needed for use in FRules and VertexValue)

Example:
prtcls(['z','b'])---->('b', 'z')

In [11]:
def prtcls(prts):
    return  tuple(sorted( prts ) )

FRules finds the feyman rules and mass matrix entries of all particles and stores then in dictionaries.
Then we can use VertexValue in order to search for these Feynman rules.

In [12]:
def FRules(TwoPointV,ThreePointV,FourPointV):
    global DictMassEntries, P3, P4
    
    DictMassEntries={}
    for i in TwoPointV:
        DictMassEntries[prtcls( map( str, i[0] ) ) ]=i[1]*(-i[2])


    P3={}
    for i in ThreePointV:
        P3[prtcls( map( str, i[0] ) ) ]=i[1]*(-sp.I*i[2])


    P4={}
    for i in FourPointV:
        P4[prtcls( map( str, i[0] ) ) ]=i[1]*(-sp.I*i[2])    

    
   

VertexValue(particle_1, particle_2, ...) gets a number of particles, and returns the  corresponding Feynman rule (or 
mass matrix entry, if the input consists of two particles).

In [13]:
def VertexValue(*particles):
    lp=len(particles)
    if lp==2:
        try:
            return DictMassEntries[ prtcls([str(particles[0] ),str(particles[1] )]) ]
        except:
            return 0

    else:
        try:
            return eval('P'+str(lp)+'[ prtcls(   map(str, particles)  ) ]'   )
        except:
            return 0
                

GetInteractions can check the indentification of the interactions, the minimization of the potential, and 
print the Feynamn rules in Frules/SUDimN/. 

The options are:

1)GetInteractions(testL=True), which checks the indentification of the interactions  

2)GetInteractions(testV=True), which checks the minimization of the Potential

3)GetInteractions(StoreVert=True), which prints files with the Feynman rules and mass matrix entries

GetInteractions() is equivalent to GetInteractions(testL=False,testV=False,StoreVert=False)

In [14]:
OPTIONS=['testL','testV','StoreVert']
DEF_OPT={'testL':False,'testV':False,'StoreVert':False}

def GetInteractions(**opts):

    if len(opts) == 0:
        print 'Using default options...'
        opts=DEF_OPT
        

            
    for i in opts:
        if not (i in OPTIONS):
            print 'invalid option '+i
            print 'availabe options: ' 
            print OPTIONS
            return 'ERR:: invalid option. Abort!'
        
    xtmp=opts.copy() 
    for i in OPTIONS:
        if not (i in opts):
            xtmp.update({i:DEF_OPT[i]})
    
    testL=xtmp['testL']
    testV=xtmp['testV']
    StoreVert=xtmp['StoreVert']
    
    if testL!=True:
        testL=False
    if testV!=True:
        testV=False    
    if StoreVert!=True:
        StoreVert=False

    if testV:
        print 'Checking vanishing of the first derivatives of the potential potential...'
        minV=np.unique(map(lambda i: \
                 sp.simplify(Deriv(V.subs(subsexpand),i ).subs(subs0).subs(subsmin) ),Fields))
        if minV==[0]:
            print 'The conditions are correct!'
        else:
            print 'The potential is not minimized correctlly...'
        


    if testL:
        global    LMassIntfinal
        print 'Checking Vertices...'
        LMassIntfinal=0
        for TypeV in TypeOfVert:
            LMassIntfinal+=np.sum([ np.product(tmpi[0])*tmpi[1] for tmpi in eval(TypeV)])
        
        if (sp.simplify(LMassIntfinal-LMassInt-constV))==0:
            print 'The interactions have been identified correctly!!'
        else:
            print 'The final Lagrangian is not the same as the initial one... (check it!)'    
           
        
    if StoreVert:
        print 'Writing Vertices (Feynman Rules and mass matrix entries)...'
        dirV='Frules'
        if not os.path.exists(dirV):
            os.makedirs(dirV)


        if not os.path.exists(dirV+"/SU" + str(dimN)):
            os.makedirs(dirV+"/SU" + str(dimN))  



        files=['TwoPoint','ThreePoint','FourPoint']


        for file in files:
            tmp = open(dirV+"/SU" + str(dimN)+ "/SU" + str(dimN)+"_" +file+"_"+gauge + ".vrt","w") 
            if file=='TwoPoint':
                factorI=-1
            else:
                factorI=-sp.I
            for i in eval(file):
                particles=str(i[0])
                
                vertex=str(factorI*i[1]*i[2])

                line='{:<40} {:<40} {:<0}'.format(particles, '|' , vertex)
                #tmp.write( particles +"|\t|"+ vertex + "\n" ) 
                tmp.write( line +'\n')
            tmp.close()        
    print 'All Done!'

In [15]:
Definitions(2,'feyn')
GetLagrangian()
IdentifyInteractions()
GetInteractions(testV=True,testL=True)

Calculating TwoPoint interactions...
Calculating ThreePoint interactions...
Calculating FourPoint interactions...
Checking vanishing of the first derivatives of the potential potential...
The conditions are correct!
Checking Vertices...
The interactions have been identified correctly!!
All Done!



<b  style="font-size: 140%;">
Keep in mind that, since the Hessian can be written in various ways (the fields can be rearreanged), 
the diagonal elements must be positive.
Among other things, as it turns out, $m_{\phi_{NN}}^2$  must be real (which is also given by the firt derivative conditions)!
</b>


In [16]:
Definitions(2,'un')
GetLagrangian()
IdentifyInteractions()
GetInteractions()
FRules(TwoPoint,ThreePoint,FourPoint)
print '--------------------------'
for i in Fields:
    tmp=sp.simplify(VertexValue(i,i))
    if tmp!=0:
        print tmp 

Calculating TwoPoint interactions...
Calculating ThreePoint interactions...
Calculating FourPoint interactions...
Using default options...
All Done!
--------------------------
lamH*v**2
lamPhi*vPhi**2
lamHPhi*v**2/2 + lamPhi*vPhi**2/2 - mPhi11**2 + mPhip11**2 - conjugate(mPhi11)**2
-4*mPhi22**2
lamHPhi*v**2/2 + lamPhi*vPhi**2/2 + mPhi11**2 + mPhip11**2 + conjugate(mPhi11)**2
